In [ ]:
import os
import pandas as pd
from pathlib import Path
import subprocess
import glob

import numpy as np
import pingouin as pg

from scipy.stats import ttest_ind
from sklearn.preprocessing import StandardScaler

from mirp import extract_features
from mirp.settings.perturbation_parameters import ImagePerturbationSettingsClass
from sklearn.mixture import GaussianMixture


Perturbation  (GTR & ITH Level)

In [ ]:
source_path='/path/to/your/data'  

In [ ]:
def assign_label(row):
    if row['image_noise_level'] == 0 and pd.isna(row['image_noise_iteration_id']) and row['image_rotation_angle'] == 0 and row['image_translation_x'] == 0 and row['image_translation_y'] == 0 and row['image_translation_z'] == 0:
        return 'r1'
    elif row['image_noise_iteration_id'] == 0 and row['image_rotation_angle'] == 0.5 and row['image_translation_x'] == 0.5 and row['image_translation_y'] == 0.5 and row['image_translation_z'] == 0.5:
        return 'r2'
    elif row['image_noise_iteration_id'] == 1 and row['image_rotation_angle'] == 0.5 and row['image_translation_x'] == 0.5 and row['image_translation_y'] == 0.5 and row['image_translation_z'] == 0.5:
        return 'r3'
    else:
        return 'other' 

In [ ]:
def check_dataset(list_volume, list_mask=None):

    if list_mask is None:  
        lists_to_check = [list_volume]
    else: 
        if len(list_volume) != len(list_mask):
            raise ValueError('There exists a mismatch between two datasets.')
        lists_to_check = [list_volume, list_mask]
    
    errors_df = pd.DataFrame(columns=['File_Path', 'Type'])
    
    for list_index, file_list in enumerate(lists_to_check):
        list_type = 'Volume' if list_index == 0 else 'Mask'
        for file_path in file_list:
            if not os.path.exists(file_path):
                
                temp_df = pd.DataFrame({'File_Path': [file_path], 'Type': [list_type]})
                errors_df = pd.concat([errors_df, temp_df], ignore_index=True)
    
    if not errors_df.empty:
        errors_df.to_excel('/path/to/errors_file_paths', index=False)
        print('Invalid file paths were found and have been saved to “errors_file_paths”.')
    else:
        print('All file paths exist.')

In [ ]:
mask_filtered_files = glob.glob(os.path.join(source_path,'perturbation','Label_ITH', '*'))

image_files = []
for file_path in mask_filtered_files:
    new_filename =os.path.join(source_path,'ICC_calculate','Image_ITH','_'.join(os.path.basename(file_path).split('_')[3:-2])+ '_image_'+os.path.basename(file_path).split('_')[-1])
    image_files.append(new_filename)

check_dataset(image_files, mask_filtered_files)

In [ ]:
def process_batch(image_batch, mask_batch, settings, output_file, append=False):
    feature_data = extract_features(
        image=image_batch,
        mask=mask_batch,
        base_discretisation_method='fixed_bin_number',
        image_modality='CT',
        base_discretisation_bin_width=16.0,
        settings=settings
    )
    combined_df = pd.concat(feature_data, ignore_index=True)
    
    write_mode = 'a' if append else 'w'
    header = not append 
    combined_df.to_csv(output_file, mode=write_mode, header=header, index=False)


settings = os.path.join(source_path,'perturbation','perturbation_test_config_settings.xml')
output_file = os.path.join(source_path,'perturbation','Perturbation_featurelevel_output.csv')

batch_size = 10  
num_batches = len(image_files) // batch_size + (1 if len(image_files) % batch_size > 0 else 0)

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = start_idx + batch_size
    image_batch = image_files[start_idx:end_idx]
    mask_batch = mask_filtered_files[start_idx:end_idx]
    process_batch(image_batch, mask_batch, settings, output_file, append=i > 0)
print('Finish for all batches.')

In [ ]:
Perturbation_featurelevel = pd.read_csv(os.path.join(source_path,'ICC_calculate','Perturbation_featurelevel_output.csv'))

Perturbation_featurelevel['reader'] = Perturbation_featurelevel.apply(assign_label, axis=1)
Perturbation_featurelevel_cleaned = Perturbation_featurelevel.dropna(subset=['cm_joint_var_d1_3d_v_mrg_fbs_w16.0', 'ngl_ldlge_d1_a0.0_3d_fbs_w16.0'])

prefixes_to_keep = ['stat_', 'ivh_', 'morph_', 'ih_', 'cm_', 'rlm_','szm_', 'dzm_','ngt_','ngl_']
columns_to_keep = ['sample_name','reader'] + [col for col in Perturbation_featurelevel.columns if any(col.startswith(prefix) for prefix in prefixes_to_keep)]

ICC_Pert = Perturbation_featurelevel[columns_to_keep].copy() 

ICC_Pert['sample_name'] = ICC_Pert['sample_name'].astype(str)
ICC_Pert['reader'] = ICC_Pert['reader'].astype(str)

icc_Pert_results =  []


In [ ]:

for column in ICC_Pert.columns[2:]:
    icc_data = ICC_Pert[['sample_name', 'reader', column]].copy()
    icc_data = icc_data.pivot(index='sample_name', columns='reader', values=column).reset_index()
    icc_data.columns.name = None  
    
    icc_data_melted = icc_data.melt(id_vars='sample_name', value_vars=['r1', 'r2', 'r3'], var_name='reader', value_name=column)
    icc = pg.intraclass_corr(data=icc_data_melted, targets='sample_name', raters='reader', ratings=column, nan_policy='omit').round(3)
    icc['feature'] = column  
    icc_Pert_results.append(icc)

per_results_df = pd.concat(icc_Pert_results, ignore_index=True)
per_results_df.to_csv('/path/to/perturbation_result_featurelevel.csv')
print('ICC calculation complete!')

per_filtered_df = per_results_df[(per_results_df['Type'] == 'ICC3k') & (per_results_df['ICC'] <= 0.75)] 
per_features_remove = per_filtered_df['feature'].unique()
len(per_features_remove)

# These features need to align with corresponding pyradiomics features.
mapping = ('/path/to/feature_correspond')
mapping = mapping.dropna()
mapping_dict = pd.Series(mapping.new_name.values, index=mapping.sample_name).to_dict()
replaced_features = []
for feature in per_features_remove:

    if pd.isna(feature):
        replaced_features.append(feature)
        continue
    
    replaced = False
    for partial_name in mapping_dict.keys():
        if partial_name in feature:
            replaced_features.append(mapping_dict[partial_name])
            replaced = True
            break
    if not replaced:
        replaced_features.append(feature)

replaced_features = np.array(replaced_features)
print(replaced_features)

In [ ]:
combined_removed_features_ITH = np.unique(replaced_features)  
len(combined_removed_features_ITH) 

final_combined_removed_features_ITH = pd.DataFrame(combined_removed_features_ITH, columns=['removed_Feature'])
final_combined_removed_features_ITH.to_excel('/path/to/Removed_features_ITH', index=False)

Feature Dimensionality Reduction--ITH Level

In [ ]:
'''
return the K value and covariance_type from the best GMM model according to the BIC
'''
def get_K(feature, K_num):    
    lowest_bic = np.infty   
    bic = []                 
    n_components_range = range(1, K_num+1)     
    cv_types = ['full']  

    for cv_type in cv_types:      
        for n_components in n_components_range:
            gmm = GaussianMixture(
                n_components=n_components, covariance_type=cv_type,random_state=0
            )
            gmm.fit(feature)
            bic.append(gmm.bic(feature))    
            if bic[-1] < lowest_bic:
                lowest_bic = bic[-1]
                best_gmm = gmm

    bic = np.array(bic)         
    return best_gmm.n_components, best_gmm.covariance_type       

'''
For each feature, assign a K value based on GMM(BIC)
Assume each patient has N features, return a list with shape (1,N) which include K for each feature
'''
def assign_K_value2features(Patient_features, K_num):   
    x, y = Patient_features.shape        
    K_list = []                  
    cov_type_list = []
    for feature_idx in range(0, y):       
        X_train = np.array(Patient_features.iloc[:, feature_idx]).reshape(-1,1)
        k, cov_type = get_K(X_train, K_num)
        K_list.append(k)
        cov_type_list.append(cov_type)

    return K_list, cov_type_list   


In [ ]:
def cluster_patient_features(Patient_features, K_list, cov_type_list):
    cluster_labels_dict = {} 
    for feature_idx, (k, cov_type) in enumerate(zip(K_list, cov_type_list)):
        feature_name = Patient_features.columns[feature_idx]  
        X_train = np.array(Patient_features.iloc[:, feature_idx]).reshape(-1, 1)
        gmm = GaussianMixture(n_components=k, covariance_type=cov_type, random_state=0)  
        gmm.fit(X_train)  
        labels = gmm.predict(X_train) 
        cluster_labels_dict[feature_name + '_Cluster'] = labels 

    cluster_labels_df = pd.DataFrame(cluster_labels_dict)
    
    return cluster_labels_df 

In [ ]:
def standardize_features(df_stand, id_columns):
    if isinstance(id_columns, str): 
        id_columns = [id_columns]

    df_stand.reset_index(drop=True, inplace=True)
    
    features_to_scale = df_stand.drop(columns=id_columns)
    
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features_to_scale)
    
    scaled_features_df = pd.DataFrame(scaled_features, columns=features_to_scale.columns)
    
    id_columns_df = df_stand[id_columns]
    
    result_df = pd.concat([id_columns_df, scaled_features_df], axis=1)
    
    return result_df, scaler

In [ ]:
df_clinical = pd.read_excel('/path/to/clinical_data')

Intratumor Feature ITH Level Cluster

In [ ]:
training_pyradiomics_ITH = pd.read_excel('/path/to/Training_feature_ITH')

prefixes_features_featurelevel_remove = final_combined_removed_features_ITH['removed_Feature'].tolist() 
training_pyradiomics_cluster_ITH= training_pyradiomics_ITH.drop(columns=prefixes_features_featurelevel_remove) 

columns_to_drop = training_pyradiomics_cluster_ITH.filter(like='original_shape').columns  
training_pyradiomics_cluster_ITH=training_pyradiomics_cluster_ITH.drop(columns=columns_to_drop)

training_inval_set = training_pyradiomics_cluster_ITH.drop(['ID','CaseID'], axis=1).apply(pd.to_numeric).reset_index(drop=True)

train_set, training_scaler = standardize_features(training_inval_set, ['ID','CaseID'])
train_set= train_set.drop(columns='CaseID') 

In [ ]:
Patients_K = pd.DataFrame()
cluster_labels_all = pd.DataFrame()
ptid = []
Patients_covtype = []
ptid_K0 = []

K_num = 5

for ptid, group in train_set.groupby('CaseID'):       
    
    try:
        analysis_data = group.drop(['CaseID'], axis=1)
        K, cov_type_list = assign_K_value2features(analysis_data.drop(['ID'], axis=1), K_num)  
        if sum(K) != 0:
            cluster_labels_df = cluster_patient_features(analysis_data.drop(['ID'], axis=1), K, cov_type_list)

            cluster_labels_df['ID'] = analysis_data['ID'].values  
            cluster_labels_df['CaseID'] = ptid

            cols = cluster_labels_df.columns.tolist()
            cols = ['ID', 'CaseID'] + [col for col in cols if col not in ['ID', 'CaseID']]
            cluster_labels_df = cluster_labels_df[cols]
            cluster_labels_all = pd.concat([cluster_labels_all, cluster_labels_df], ignore_index=True, axis=0)

            K_new = [ptid] + K  
            K_new_df=pd.DataFrame([K_new], columns=['CaseID'] +  list(train_set.columns.drop(['ID','CaseID'])))
            Patients_K = pd.concat([Patients_K, K_new_df], ignore_index=True, axis=0)
            # print(f'{ptid} is done')
        else:
            ptid_K0.append(ptid)
            print(f'something wrong: K is 0 for patient {ptid}')
    except Exception as e:
        print(f'Error: something wrong about K for patient {ptid}, Error: {e}')
    finally:
        Patients_covtype.append((ptid, cov_type_list))


In [ ]:
Patients_K_IDrep = pd.merge(training_pyradiomics_ITH[['CaseID']], Patients_K, on='CaseID', how='left')
Patients_K_IDrep.to_excel('/path/to/training_ITH_feature_vector')
#cluster_labels_all.to_excel('/path/to/training_ITH_feature_cluster')

Pearson correlation analyses

In [ ]:
# # Pearson Correlation Coefficient (PCC) to remove highly correlated features
corr_matrix = Patients_K.drop(['CaseID'], axis=1).corr()  

high_corr_var=np.where(corr_matrix>0.75)  

to_remove = set()  

for var in zip(*high_corr_var):  
    if var[0] != var[1] and var[0] not in to_remove: 
        to_remove.add(var[1]) 

df_filtered = Patients_K.drop(columns=Patients_K.columns[list(to_remove)]) 
pcc_features_remove = corr_matrix.columns[list(to_remove)]
len(pcc_features_remove)  

Independent T-test

In [ ]:
df_filtered = pd.merge(df_filtered, training_pyradiomics_ITH[['CaseID']], on='CaseID', how='left')
df_Ttest = pd.merge(df_filtered, df_clinical[['CaseID', 'ORR_RECIST1.1']], on='CaseID', how='left')

columns = list(df_Ttest.columns)
columns.remove('ORR_RECIST1.1')
columns.insert(1, 'ORR_RECIST1.1')
df_Ttest = df_Ttest[columns] 

In [ ]:
group_0 = df_Ttest[df_Ttest['ORR_RECIST1.1'] == 0]  
group_1 = df_Ttest[df_Ttest['ORR_RECIST1.1'] == 1] 

features = df_Ttest.columns[2:]

significant_features = []
p_values = []
for feature in features:
    stat, p_value = ttest_ind(group_0[feature], group_1[feature])
    p_values.append(p_value)  
    if p_value < 0.05: 
        significant_features.append(feature)

print(len(significant_features), 'Significant features:', significant_features)

p_values_df = pd.DataFrame({'Feature': features, 'P-Value': p_values})
print(p_values_df)  

In [ ]:
final_features_patientlevel = pd.DataFrame(significant_features, columns=['Feature'])
final_features_patientlevel.to_excel('/path/to/final_features_ITH_list')

Internal validation set cluster

In [ ]:
inval_set_pyradiomics_ITH = pd.read_excel('/path/to/Inval_feature_ITH')
columns_to_keep = ['CaseID','ID'] + [col for col in inval_set_pyradiomics_ITH.columns if col in significant_features]

inval_pyradiomics_ITH = inval_set_pyradiomics_ITH[columns_to_keep].copy()  

df_numeric_inval = inval_pyradiomics_ITH.drop(['ID','CaseID'], axis=1).apply(pd.to_numeric)
inval_pyradiomics_cluster_test_ITH = df_numeric_inval.reset_index(drop=True)

features_to_scale_validation_ITH = inval_pyradiomics_cluster_test_ITH.drop(['CaseID','ID'], axis=1)
scaled_features_validation_ITH  = training_scaler.transform(features_to_scale_validation_ITH)

scaled_features_validation_df = pd.DataFrame(scaled_features_validation_ITH, columns=features_to_scale_validation_ITH.columns)
df_validation_standardized = pd.concat([inval_pyradiomics_cluster_test_ITH[['CaseID', 'ID']], scaled_features_validation_df], axis=1)

In [ ]:
Patients_K_exter=  Patients_K[['CaseID'] +[col for col in Patients_K.columns if col in significant_features]].copy()
max_K_values = Patients_K_exter.iloc[:, 1:].max()

cluster_labels_validation_all = df_validation_standardized
ptid_K0_test = []
Patients_K_test = pd.DataFrame()
cluster_labels_all_test = pd.DataFrame()
ptid = []
Patients_covtype_test = []


for ptid, group in cluster_labels_validation_all.groupby('CaseID'):
    try:
        analysis_data = group.drop(['CaseID'], axis=1)
        K, cov_type_list = assign_K_value2features(analysis_data.drop(['ID'], axis=1), K_num) 
        K_limited = K

        if sum(K_limited) != 0:
            cluster_labels_df = cluster_patient_features(analysis_data.drop(['ID'], axis=1), K_limited, cov_type_list)

            cluster_labels_df['ID'] = analysis_data['ID'].values 
            cluster_labels_df['CaseID'] = ptid

            cols = cluster_labels_df.columns.tolist()
            cols = ['ID', 'CaseID'] + [col for col in cols if col not in ['ID', 'CaseID']]
            cluster_labels_df = cluster_labels_df[cols]
            cluster_labels_all_test = pd.concat([cluster_labels_all_test, cluster_labels_df], ignore_index=True, axis=0)

            K_new = [ptid] + K_limited  
            K_new_df=pd.DataFrame([K_new], columns=['CaseID'] +  list(cluster_labels_validation_all.columns.drop(['ID','CaseID'])))
            Patients_K_test = pd.concat([Patients_K_test, K_new_df], ignore_index=True, axis=0)
        else:
            ptid_K0_test.append(ptid)
            print(f'something wrong: K is 0 for patient {ptid}')
    except Exception as e:
        print(f'Error: something wrong about K for patient {ptid}, Error: {e}')
    finally:
        Patients_covtype_test.append((ptid, cov_type_list))


In [ ]:
Patients_K_inval_IDrep = pd.merge(inval_set_pyradiomics_ITH[['CaseID']], Patients_K_test, on='CaseID', how='left')
Patients_K_inval_IDrep.to_excel('/path/to/Inval_ITH_feature_vector')
#cluster_labels_all_test.to_excel('/path/to/Inval_ITH_feature_cluster')

External test set cluster

In [ ]:
testing_set_pyradiomics_ITH = pd.read_excel('/path/to/Testing_feature_ITH')
columns_to_keep = ['CaseID','ID'] + [col for col in testing_set_pyradiomics_ITH.columns if col in significant_features] 

testing_pyradiomics_ITH = testing_set_pyradiomics_ITH[columns_to_keep].copy() 

df_numeric_test = testing_pyradiomics_ITH.drop(['ID','CaseID'], axis=1).apply(pd.to_numeric)
testing_pyradiomics_cluster_test_ITH = df_numeric_test.reset_index(drop=True)

features_to_scale_test_ITH = testing_pyradiomics_cluster_test_ITH.drop(['CaseID','ID'], axis=1)
scaled_features_test_ITH  = training_scaler.transform(features_to_scale_test_ITH)

scaled_features_test_df = pd.DataFrame(scaled_features_test_ITH, columns=features_to_scale_test_ITH.columns)
df_test_standardized = pd.concat([testing_pyradiomics_cluster_test_ITH[['CaseID', 'ID']], scaled_features_test_df], axis=1)

In [ ]:
Patients_K_exter=  Patients_K[['CaseID'] +[col for col in Patients_K.columns if col in significant_features]].copy()
max_K_values = Patients_K_exter.iloc[:, 1:].max()

cluster_labels_test_all = df_test_standardized
ptid_K0_test = []
Patients_K_test = pd.DataFrame()
cluster_labels_all_test = pd.DataFrame()
ptid = []
Patients_covtype_test = []

for ptid, group in cluster_labels_test_all.groupby('CaseID'):
    try:
        analysis_data = group.drop(['CaseID'], axis=1)
        K, cov_type_list = assign_K_value2features(analysis_data.drop(['ID'], axis=1), K_num) 
        K_limited = K

        if sum(K_limited) != 0:
            cluster_labels_df = cluster_patient_features(analysis_data.drop(['ID'], axis=1), K_limited, cov_type_list)

            cluster_labels_df['ID'] = analysis_data['ID'].values  
            cluster_labels_df['CaseID'] = ptid

            cols = cluster_labels_df.columns.tolist()
            cols = ['ID', 'CaseID'] + [col for col in cols if col not in ['ID', 'CaseID']]
            cluster_labels_df = cluster_labels_df[cols]
            cluster_labels_all_test = pd.concat([cluster_labels_all_test, cluster_labels_df], ignore_index=True, axis=0)

            K_new = [ptid] + K_limited  
            K_new_df=pd.DataFrame([K_new], columns=['CaseID'] +  list(cluster_labels_validation_all.columns.drop(['ID','CaseID'])))
            Patients_K_test = pd.concat([Patients_K_test, K_new_df], ignore_index=True, axis=0)
        else:
            ptid_K0_test.append(ptid)
            print(f'something wrong: K is 0 for patient {ptid}')
    except Exception as e:
        print(f'Error: something wrong about K for patient {ptid}, Error: {e}')
    finally:
        Patients_covtype_test.append((ptid, cov_type_list))


In [ ]:
Patients_K_test_IDrep = pd.merge(testing_pyradiomics_ITH[['CaseID']], Patients_K_test, on='CaseID', how='left')
Patients_K_test_IDrep.to_excel('/path/to/testing_ITH_feature_vector')
#cluster_labels_all_test.to_excel('/path/to/testing_ITH_feature_cluster')

TCIA-TCGA数据集

In [ ]:
TCIA_TCGA_set_pyradiomics_ITH = pd.read_excel('/path/to/TCIA_TCGA_feature_ITH')
columns_to_keep = ['CaseID','ID'] + [col for col in TCIA_TCGA_set_pyradiomics_ITH.columns if col in significant_features]

TCIA_TCGA_pyradiomics_ITH = TCIA_TCGA_set_pyradiomics_ITH[columns_to_keep].copy()  

df_numeric_TCGA = TCIA_TCGA_pyradiomics_ITH.drop(['ID','CaseID'], axis=1).apply(pd.to_numeric)
TCIA_TCGA_pyradiomics_cluster_test_ITH = df_numeric_TCGA.reset_index(drop=True)

features_to_scale_TCIA_TCGA_ITH = TCIA_TCGA_pyradiomics_cluster_test_ITH.drop(['CaseID','ID'], axis=1)
scaled_features_TCIA_TCGA_ITH  = training_scaler.transform(features_to_scale_TCIA_TCGA_ITH)


scaled_features_TCIA_TCGA_df = pd.DataFrame(scaled_features_TCIA_TCGA_ITH, columns=features_to_scale_TCIA_TCGA_ITH.columns)
df_TCIA_TCGA_standardized = pd.concat([TCIA_TCGA_pyradiomics_cluster_test_ITH[['CaseID', 'ID']], scaled_features_TCIA_TCGA_df], axis=1)

In [ ]:
cluster_labels_TCIA_TCGA_all = df_TCIA_TCGA_standardized
ptid_K0_TCIA_TCGA = []
Patients_K_TCIA_TCGA = pd.DataFrame()
cluster_labels_all_TCIA_TCGA = pd.DataFrame()
ptid = []
Patients_covtype_TCIA_TCGA = []

for ptid, group in cluster_labels_TCIA_TCGA_all.groupby('CaseID'):
    try:
        analysis_data = group.drop(['CaseID'], axis=1)
        K, cov_type_list = assign_K_value2features(analysis_data.drop(['ID'], axis=1), K_num) 
        K_limited = K

        if sum(K_limited) != 0:
            cluster_labels_df = cluster_patient_features(analysis_data.drop(['ID'], axis=1), K_limited, cov_type_list)

            cluster_labels_df['ID'] = analysis_data['ID'].values  
            cluster_labels_df['CaseID'] = ptid

            cols = cluster_labels_df.columns.tolist()
            cols = ['ID', 'CaseID'] + [col for col in cols if col not in ['ID', 'CaseID']]
            cluster_labels_df = cluster_labels_df[cols]
            cluster_labels_all_TCIA_TCGA = pd.concat([cluster_labels_all_TCIA_TCGA, cluster_labels_df], ignore_index=True, axis=0)

            K_new = [ptid] + K_limited   
            K_new_df=pd.DataFrame([K_new], columns=['CaseID'] +  list(cluster_labels_validation_all.columns.drop(['ID','CaseID'])))
            Patients_K_TCIA_TCGA = pd.concat([Patients_K_TCIA_TCGA, K_new_df], ignore_index=True, axis=0)
        else:
            ptid_K0_TCIA_TCGA.append(ptid)
            print(f'something wrong: K is 0 for patient {ptid}')
    except Exception as e:
        print(f'Error: something wrong about K for patient {ptid}, Error: {e}')
    finally:
        Patients_covtype_test.append((ptid, cov_type_list))


In [ ]:
Patients_K_TCIA_TCGA_IDrep = pd.merge(TCIA_TCGA_pyradiomics_ITH[['CaseID']], Patients_K_TCIA_TCGA, on='CaseID', how='left')
Patients_K_TCIA_TCGA_IDrep.to_excel('/path/to/TCIA_TCGA_ITH_feature_vector')
# cluster_labels_all_TCIA_TCGA.to_excel('/path/to/TCIA_TCGA_ITH_feature_cluster')